In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os

AI6Hz_normal, AI6Hz_anomalies = [], []
root_folder_norm = 'drive/MyDrive/data/data_Marcel/AI6Hz_normal/'
for root, dirs, files in os.walk(root_folder_norm, topdown=False):
    for name in files:
      if name.endswith('.txt'):
        AI6Hz_normal.append(name)
root_folder_anom = 'drive/MyDrive/data/data_Marcel/AI6Hz_anomalies/'
for root, dirs, files in os.walk(root_folder_anom, topdown=False):
    for name in files:
      if name.endswith('.txt'):
        AI6Hz_anomalies.append(name)

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from os.path import dirname, join as pjoin
import scipy.io as sio
import scipy
def main2(s_train, s_evaluate, ts = None, batch_size = 256):
  from sklearn.preprocessing import   MinMaxScaler
  import datetime
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import LSTM
  from tensorflow.keras.layers import Dense
  from tensorflow.keras.layers import RepeatVector
  from tensorflow.keras.layers import TimeDistributed
  from tensorflow.keras.utils import plot_model
  from sklearn import preprocessing
  from keras import backend as K 
  import random
  scaler = preprocessing.StandardScaler()
  s_train = scaler.fit_transform(s_train.reshape(-1, 1))
  s_evaluate = scaler.fit_transform(s_evaluate.reshape(-1, 1))
  seed_value= 1
  os.environ['PYTHONHASHSEED'] = str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.random.set_seed(seed_value)
  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads = 1, inter_op_parallelism_threads = 1)
  sess = tf.compat.v1.Session(graph =  tf.compat.v1.get_default_graph(), config = session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  start_time = datetime.datetime.now()
  df_daily_jumpsup = s_evaluate
  df_small_noise = s_train #s_train[0:len(df_daily_jumpsup)] 
  training_mean = np.mean(df_small_noise)
  training_std = np.std(df_small_noise)
  #scaler = StandardScaler()
  #df_training_value = scaler.fit_transform(df_small_noise.reshape(-1, 1))
  df_training_value = df_small_noise.reshape(-1, 1)
  print("Number of training samples:", len(df_training_value))
  TIME_STEPS =  np.floor(np.std(signal_test) * 5387).astype('int64') if ts == None else ts
  # Generated training sequences for use in the model.
  def create_sequences(values, time_steps = TIME_STEPS):
      output = []
      for i in range(TIME_STEPS, len(values)- TIME_STEPS):
          output.append(values[i : (i + time_steps)])
      return np.stack(output)
  x_train = create_sequences(df_training_value)
  x_train = x_train.reshape(x_train.shape[0], TIME_STEPS, 1)
  print(x_train.shape)
  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)
  model = Sequential()
  model.add(LSTM(100, activation = 'tanh', recurrent_dropout = 0, input_shape = (x_train.shape[1],x_train.shape[2])
  ,return_sequences = True))
  model.add(LSTM(64, activation = 'tanh', recurrent_dropout = 0, return_sequences = False))
  model.add(keras.layers.Dropout(rate = 0.2))
  model.add(RepeatVector(x_train.shape[1]))
  model.add(keras.layers.Dropout(rate = 0.2))
  model.add(LSTM(64, activation = 'tanh', recurrent_dropout = 0, return_sequences = True))
  model.add(LSTM(100, activation = 'tanh', recurrent_dropout = 0, return_sequences = True))
  model.add(TimeDistributed(Dense(1)))
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001), loss = "mse")
  model.summary()
  history = model.fit(
      x_train,
      x_train,
      epochs = 100,
      batch_size = batch_size,
      validation_split = 0.25,
      callbacks = [
          keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 15, mode = "min")
      ,[tensorboard_callback]],
      verbose = 0
      )
  x_train_pred = model.predict(x_train)
  train_mae_loss = np.mean(np.abs(x_train_pred - x_train), axis = 1)
  threshold = np.abs(np.percentile(train_mae_loss,95)+np.percentile(train_mae_loss,5))
  print("Reconstruction error threshold: ", threshold)
  #df_test_value = scaler.fit_transform(df_daily_jumpsup.reshape(-1, 1))
  df_test_value = df_daily_jumpsup.reshape(-1, 1)
  x_test = create_sequences(df_test_value)
  x_test = x_test.reshape(x_test.shape[0], TIME_STEPS, 1)
  print("Test input shape: ", x_test.shape)
  # Get test MAE loss.
  x_test_pred = model.predict(x_test)
  test_mae_loss =np.mean(np.abs(x_test_pred - x_test), axis = 1)
  test_mae_loss = test_mae_loss.reshape((-1))
  anomalies = test_mae_loss > threshold
  anomalous_data_indices = []
  for data_idx in range(TIME_STEPS - 1, len(x_test) - TIME_STEPS + 1):
      if sum(anomalies[data_idx - TIME_STEPS + 1 : data_idx]) >= TIME_STEPS / 2:
          anomalous_data_indices.append(data_idx)
  anomalies[data_idx - TIME_STEPS + 1 : data_idx]
  df1 = s_evaluate
  df_subset = df1[anomalous_data_indices]
  end_time =  datetime.datetime.now()
  exec_time = end_time - start_time
  print(f'execution time {exec_time}')
  return x_train, x_test, x_test_pred, train_mae_loss, test_mae_loss, anomalous_data_indices

def plot2_1(df_training_value, df_test_value, x_test_pred, train_mae_loss, test_mae_loss, anomalous_data_indices, title = None):
  import plotly.graph_objects as go
  from plotly.subplots import make_subplots
  import plotly
  from sklearn import preprocessing
  scaler = preprocessing.StandardScaler()
  fig = make_subplots(rows = 5, cols = 2, horizontal_spacing = 0.1, vertical_spacing = 0.03,
  specs=[[{}, {"rowspan": 2}],
          [{}, None],
          [{},None],
          [{}, {"rowspan": 2}],
          [{},None]],
          print_grid = True,
          shared_xaxes= True,
          column_widths = (0.7, 0.3))
  fig.add_trace(go.Scatter( y = df_training_value,
                          line_shape = 'linear', name = 'Train', marker_color = '#000000'), row = 1, col = 1)
  
  fig.add_trace(go.Histogram(x = train_mae_loss.flatten(),
                            nbinsx = 30, marker_color = '#58D68D', name = 'train mae loss', opacity = 0.5), row = 1, col = 2)
  
  fig.add_trace(go.Histogram(x = test_mae_loss.flatten(),
                            nbinsx = 30, marker_color = '#D35400', name = 'test mae loss', opacity = 0.6), row = 4, col = 2)
    
  fig.add_trace(go.Scatter(y = df_test_value,
                          line_shape = 'linear', name = 'Test', line = dict(width = 2, color = '#000000')), row = 2, col = 1)

  fig.add_trace(go.Scatter(y = x_test_pred,
                          line_shape = 'linear', name = 'RNN pred', line = dict(width = 2, color = '#000000')), row = 3, col = 1)
  
  fig.add_trace(go.Scatter(y = test_mae_loss,
                          line_shape = 'linear', name = 'Error', line = dict(width = 2, color = '#990000')), row = 4, col = 1)
  
  fig.add_trace(go.Scatter( y = df_test_value,
                          line_shape = 'linear', name = 'Signal', marker_color = '#000000'), row = 5, col = 1)
  B = list(anomalous_data_indices)
  A = []
  C = []
  #next index
  windows_size = 2
  N_index = windows_size
  #actual index
  A_index = 0
  for i in range(len(B) - windows_size):
    if (B[N_index] - B[A_index]) / B[N_index] < 0.02:
      C.append(B[A_index])
      N_index += 1
      A_index += 1
    else:
      C.append(B[A_index])
      A.append(C)
      N_index = N_index + 1
      A_index = A_index + 1
      C = []
      # C.append(B[N_index])

  flat_list = [item for sublist in A for item in sublist]
  A.append([item for item in B if item not in flat_list])
  condition = 0
  for sublist in A:
    fig.add_trace(go.Scatter(x = sublist, y = df_test_value[sublist],
                            name = 'Anomaly', showlegend = True if condition < 1 == True else False , mode='markers',
                            marker=dict(size=3,
                            color='#990000'), opacity = 0.8),
                            row = 5, col = 1)
    condition +=1
  fig.add_trace(go.Scatter(x = np.linspace(0,len(test_mae_loss),20000), y = np.linspace(np.max(train_mae_loss),np.max(train_mae_loss),20000),
                            line_shape = 'linear', name = 'Threshold',
                            line = dict(width = 2, color = '#138D75'),opacity = 0.7,
                            hovertemplate = f'Threshold: {np.max(train_mae_loss)}'
                            ), row = 4, col = 1)
  
  fig.update_layout(shapes = [
      dict(
        type = 'line',
        yref = 'paper', y0 = 0, y1 = 0.4 - 0.015,
        xref = 'x6', x0 = np.max(train_mae_loss), x1 = np.max(train_mae_loss),
        line = dict(color = '#990000', width = 1.5)
              )     
  ],
  plot_bgcolor='rgb(255, 255, 255)',
  #height = 600, width = 'device-width',
  title = title
  )

  fig.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = 'Gray',
                  zeroline = True, zerolinecolor = '#000',row = 1, col = 2,
                  nticks = 5
                  )
  fig.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = 'Gray',
                  zeroline = True, zerolinecolor = '#000',row = 4, col = 2,
                  nticks = 5
                  )

  fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = 'Gray',
                  zeroline = False, zerolinewidth = 2, zerolinecolor = '#000', 
                  row = 1, col = 2, nticks = 5
                  )
  fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = 'Gray',
                  zeroline = False, zerolinewidth = 2, zerolinecolor = '#000', 
                  row = 4, col = 2, nticks = 5
                  )

  fig.add_annotation(
        x = np.abs(np.percentile(train_mae_loss,95)+np.percentile(train_mae_loss,5)),
        y = -0.7,
        xref="x2",
        yref="paper",
        text= '{0:.2f}'.format(np.max(train_mae_loss)),
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=11,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#990000",
        borderwidth=1,
        borderpad=2,
        bgcolor="#990000",
        opacity=1
        , row = 1, col = 2
        )
  # fig.add_annotation(
  #       x = -0.5,
  #       y = np.max(train_mae_loss)*2,
  #       xref= "paper",
  #       yref="y",
  #       text= 'Threshold: {0:.2f}'.format(np.max(train_mae_loss)),
  #       showarrow=False,
  #       font=dict(
  #           family="Courier New, monospace",
  #           size=11,
  #           color="#ffffff"
  #           ),
  #       align="center",
  #       bordercolor="#138D75",
  #       borderwidth=1,
  #       borderpad=2,
  #       bgcolor="#138D75",
  #       opacity=1
  #       , row = 3, col = 2
  #       )
  fig.add_annotation(
        x = np.max(train_mae_loss),
        y = 1,
        xref="x6",
        yref="paper",
        text= '{0:.2f}'.format(np.max(train_mae_loss)),
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=11,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#990000",
        borderwidth=1,
        borderpad=2,
        bgcolor="#990000",
        opacity=1
        , row = 4, col = 2
        )

  return fig

In [ ]:
import plotly
import datetime
ts = 190
for signal in AI6Hz_normal:
  if signal.replace('n', 'a') in AI6Hz_anomalies:
    signal_train = noise_reduct(signal_miliseconds(root_folder_norm, signal))
    signal_test  = noise_reduct(signal_miliseconds(root_folder_anom, signal.replace('n', 'a')))
    asize= [signal_train.shape if signal_train.shape < signal_test.shape else signal_test.shape][0][0]
    signal_train = signal_train[ : asize]
    signal_test  = signal_test[  : asize]
    df_training_value, df_test_value, x_test_pred, train_mae_loss, test_mae_loss, anomalous_data_indices = main2(signal_train, signal_test, ts = ts)
    fig = plot2_1(np.squeeze(df_training_value)[:,0], np.squeeze(df_test_value)[:,0], np.squeeze(x_test_pred)[:,0], train_mae_loss, test_mae_loss, anomalous_data_indices, title = str(signal) + f'  TimeSteps = {ts}')
    plotly.offline.plot(fig, filename='drive/MyDrive/htmlplots/'+str(signal.split('.')[0])+f'ts{ts}_{datetime.datetime.now().strftime("%a, %d %B %Y %H:%M:%S")}.html')
  print("done!")